# ideas

- Have it create 3 objects that represent all the songs. 
- Get the general vibe
- Feed to Flux replicate

In [10]:
import requests
import base64
import json
import os
from statistics import mean

# Replace these with your actual Client ID and Client Secret
CLIENT_ID = os.environ.get('SPOTIFY_CLIENT_ID')
CLIENT_SECRET = os.environ.get('SPOTIFY_CLIENT_SECRET')

In [3]:
client_creds = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_creds_b64 = base64.b64encode(client_creds.encode())

# Token URL
token_url = 'https://accounts.spotify.com/api/token'

# Request Body Parameters
token_data = {
    'grant_type': 'client_credentials'
}

# Request Headers
token_headers = {
    'Authorization': f'Basic {client_creds_b64.decode()}'
}

In [5]:
r = requests.post(token_url, data=token_data, headers=token_headers)

if r.status_code == 200:
    print("Authentication successful!")
    token = r.json()['access_token']
    
    # Now let's try to get info about a track
    track_id = '11dFghVXANMlKmJXsNCbNl'  # This is the Spotify ID for "Cut To The Feeling" by Carly Rae Jepsen
    track_url = f'https://api.spotify.com/v1/tracks/{track_id}'
    
    track_headers = {
        'Authorization': f'Bearer {token}'
    }
    
    track_response = requests.get(track_url, headers=track_headers)
    
    if track_response.status_code == 200:
        track_data = track_response.json()
        print(f"Successfully retrieved data for the track: {track_data['name']} by {track_data['artists'][0]['name']}")
    else:
        print(f"Failed to retrieve track data. Status code: {track_response.status_code}")
        print(f"Error message: {track_response.text}")
else:
    print(f"Authentication failed. Status code: {r.status_code}")
    print(f"Error message: {r.text}")

Authentication successful!
Successfully retrieved data for the track: Cut To The Feeling by Carly Rae Jepsen


In [6]:
PLAYLIST_ID = '37i9dQZF1DX0XUsuxWHRQd'

# Authentication
def get_token():
    auth_string = f"{CLIENT_ID}:{CLIENT_SECRET}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = requests.post(url, headers=headers, data=data)
    json_result = result.json()
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

# Get playlist tracks
def get_playlist_tracks(token, playlist_id):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = get_auth_header(token)
    
    tracks = []
    while url:
        result = requests.get(url, headers=headers)
        json_result = result.json()
        
        for item in json_result['items']:
            track_name = item['track']['name']
            artist_name = item['track']['artists'][0]['name']
            tracks.append(f"{track_name} by {artist_name}")
        
        url = json_result.get('next')  # Get the next page URL, if it exists

    return tracks

# Main execution
token = get_token()
tracks = get_playlist_tracks(token, PLAYLIST_ID)

print(f"Tracks in the playlist (total: {len(tracks)}):")
for i, track in enumerate(tracks, 1):
    print(f"{i}. {track}")

Tracks in the playlist (total: 50):
1. It's Up (feat. Young Thug & 21 Savage) by Drake
2. Parking Lot by Mustard
3. FIELD TRIP by ¥$
4. BAND4BAND (feat. Lil Baby) by Central Cee
5. Not Like Us by Kendrick Lamar
6. one of wun by Gunna
7. KEHLANI by Jordan Adetunji
8. ATTITUDE (FEAT. CHARLIE WILSON & CASH COBAIN) by Don Toliver
9. Tobey (feat. Big Sean and BabyTron) by Eminem
10. Yeah Glo! by GloRilla
11. Like That by Future
12. on one tonight by Gunna
13. BROTHER STONE (FEAT. KODAK BLACK) by Don Toliver
14. HEAT STICK by Shoreline Mafia
15. Buffalo Freestyle (feat. Drake) by Benny The Butcher
16. Boogieman by EBK Jaaybo
17. MALIBU - WITH TEE GRIZZLEY & G HERBO by YG
18. Type Shit by Future
19. Billion Streams Freestyle by Central Cee
20. Talk My Shit by BossMan Dlow
21. John Wayne (feat. Metro Boomin) by Young Nudy
22. Hate Me by Lil Yachty
23. HIGHJACK by A$AP Rocky
24. Together Forever (Big Sean & The Alchemist) by Big Sean
25. Cinderella by Future
26. TASTE by Lil Tecca
27. prada dem

In [12]:
PLAYLIST_ID = '37i9dQZF1DX0XUsuxWHRQd'

# Authentication
def get_token():
    auth_string = f"{CLIENT_ID}:{CLIENT_SECRET}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = requests.post(url, headers=headers, data=data)
    json_result = result.json()
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

# Get playlist tracks
def get_playlist_tracks(token, playlist_id):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = get_auth_header(token)
    
    tracks = []
    while url:
        result = requests.get(url, headers=headers)
        json_result = result.json()
        
        for item in json_result['items']:
            track = item['track']
            tracks.append({
                'id': track['id'],
                'name': track['name'],
                'artist': track['artists'][0]['name']
            })
        
        url = json_result.get('next')  # Get the next page URL, if it exists

    return tracks

# Get audio features for tracks
def get_audio_features(token, track_ids):
    url = f"https://api.spotify.com/v1/audio-features"
    headers = get_auth_header(token)
    params = {'ids': ','.join(track_ids)}
    
    result = requests.get(url, headers=headers, params=params)
    return result.json()['audio_features']

# Analyze vibe based on audio features
def analyze_vibe(features):
    valence = features['valence']
    energy = features['energy']
    danceability = features['danceability']
    tempo = features['tempo']
    
    vibe = []
    if valence > 0.6:
        vibe.append("positive")
    elif valence < 0.4:
        vibe.append("melancholic")
    
    if energy > 0.7:
        vibe.append("energetic")
    elif energy < 0.3:
        vibe.append("calm")
    
    if danceability > 0.7:
        vibe.append("danceable")
    
    if tempo > 120:
        vibe.append("upbeat")
    elif tempo < 80:
        vibe.append("slow")
    
    return ", ".join(vibe) if vibe else "neutral"

# Main execution
token = get_token()
tracks = get_playlist_tracks(token, PLAYLIST_ID)

# Get audio features for all tracks
track_ids = [track['id'] for track in tracks]
audio_features = get_audio_features(token, track_ids)

# Calculate averages
avg_valence = mean(feature['valence'] for feature in audio_features)
avg_energy = mean(feature['energy'] for feature in audio_features)

print(f"Tracks in the playlist (total: {len(tracks)}):")
for i, (track, features) in enumerate(zip(tracks, audio_features), 1):
    vibe = analyze_vibe(features)
    print(f"{i}. {track['name']} by {track['artist']}")
    print(f"   Vibe: {vibe}")
    print(f"   Valence: {features['valence']:.2f}, Energy: {features['energy']:.2f}")
    print()

print(f"Playlist Averages:")
print(f"Average Valence: {avg_valence:.2f}")
print(f"Average Energy: {avg_energy:.2f}")

# Analyze overall playlist vibe
playlist_vibe = []
if avg_valence > 0.6:
    playlist_vibe.append("positive")
elif avg_valence < 0.4:
    playlist_vibe.append("melancholic")
else:
    playlist_vibe.append("mix of positive and melancholic")

if avg_energy > 0.7:
    playlist_vibe.append("high-energy")
elif avg_energy < 0.3:
    playlist_vibe.append("low-energy")
else:
    playlist_vibe.append("moderate energy")

print(f"Overall Playlist Vibe: {', '.join(playlist_vibe)}")

Tracks in the playlist (total: 50):
1. It's Up (feat. Young Thug & 21 Savage) by Drake
   Vibe: melancholic, energetic, danceable, upbeat
   Valence: 0.30, Energy: 0.72

2. Parking Lot by Mustard
   Vibe: danceable
   Valence: 0.41, Energy: 0.56

3. FIELD TRIP by ¥$
   Vibe: danceable, upbeat
   Valence: 0.46, Energy: 0.69

4. BAND4BAND (feat. Lil Baby) by Central Cee
   Vibe: positive, energetic, danceable, upbeat
   Valence: 0.89, Energy: 0.76

5. Not Like Us by Kendrick Lamar
   Vibe: melancholic, danceable
   Valence: 0.21, Energy: 0.47

6. one of wun by Gunna
   Vibe: melancholic, danceable, upbeat
   Valence: 0.29, Energy: 0.45

7. KEHLANI by Jordan Adetunji
   Vibe: melancholic, upbeat
   Valence: 0.35, Energy: 0.63

8. ATTITUDE (FEAT. CHARLIE WILSON & CASH COBAIN) by Don Toliver
   Vibe: upbeat
   Valence: 0.43, Energy: 0.66

9. Tobey (feat. Big Sean and BabyTron) by Eminem
   Vibe: melancholic, upbeat
   Valence: 0.21, Energy: 0.66

10. Yeah Glo! by GloRilla
   Vibe: melanchol